# Assistant retrievel

* O recurso de retrieval dos Assistants da OpenAI (atualmente chamado de file_search) é uma funcionalidade avançada que permite aos modelos de IA acessar informações externas para oferecer respostas mais precisas e contextualizadas.
* Com o retrieval, o assistente é capaz de buscar dados em documentos fornecidos pelo usuário, como PDFs, arquivos de texto e outros formatos e utilizar essas informações diretamente para construir respostas mais relevantes, fundamentadas e específicas.
* No caso deste projeto, o retrieval será utilizado para pesquisar o conteúdo de um arquivo PDF enviado.

In [1]:
import openai
from dotenv import load_dotenv
import os
import time

In [2]:
# Acessando a API

load_dotenv()
secret_key = os.getenv('OPEN_API_KEY')

In [3]:
# Instanciando 

client = openai.Client(api_key=secret_key)

In [4]:
# 1. Faz upload do PDF
uploaded_file = client.files.create(
    file=open("files/LLM.pdf", "rb"),
    purpose="assistants"
)
print(f"Arquivo enviado: {uploaded_file.id}")

Arquivo enviado: file-67hcmYSVda11JXUV6xKByV


In [5]:
# 2. Cria o Assistant (sem arquivos) com ferramenta file_search
assistant = client.beta.assistants.create(
    name="Tutor de Apostilas",
    instructions="Responda às perguntas usando as informações da apostila fornecida.",
    model="gpt-4-turbo",
    tools=[{"type": "file_search"}]
)
print(f"Assistant criado: {assistant.id}")

Assistant criado: asst_tk1MIkqJkwwrRf8Sl0fpNhBi


In [6]:
# 3. Cria o Thread já enviando a primeira mensagem e anexando o arquivo
thread = client.beta.threads.create(
    messages=[
        {
            "role": "user",
            "content": "Quais são os principais temas abordados nessa apostila?",
            # Aqui vinculamos o PDF ao thread via attachments
            "attachments": [
                {
                    "file_id": uploaded_file.id,
                    "tools": [{"type": "file_search"}]
                }
            ],
        }
    ]
)
print(f"Thread criada: {thread.id}")

Thread criada: thread_9fzgqWNiw1dUux2l9GYBk1rj


In [7]:
# 4. Executa o assistant nesse thread
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id
)
print(f"Run criada: {run.id}")

Run criada: run_tTZMMT0VcphZKOVASD5xW3jf


In [8]:
# 5. Aguarda a run completar
while True:
    status = client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run.id
    ).status
    if status == "completed":
        print("Run finalizada!")
        break
    elif status == "failed":
        raise RuntimeError("A execução falhou.")
    else:
        print(f"Status: {status}... aguardando")
        time.sleep(2)

Status: queued... aguardando
Status: in_progress... aguardando
Status: in_progress... aguardando
Status: in_progress... aguardando
Status: in_progress... aguardando
Status: in_progress... aguardando
Run finalizada!


In [9]:
# 6. Lista e imprime as mensagens do assistant
messages = client.beta.threads.messages.list(thread_id=thread.id)
for m in messages.data:
    if m.role == "assistant":
        # mensagem formatada de acordo com o schema v2
        print("\nResposta do Assistant:")
        print(m.content[0].text.value)


Resposta do Assistant:
A apostila aborda principalmente os Grandes Modelos de Linguagem (LLM), cobrindo uma variedade de temas como:

1. **Introdução e Definição de LLMs:** Explicações sobre o que são os LLMs, suas funções e importância no campo da inteligência artificial, especificamente no processamento de grandes volumes de dados de linguagem natural【4:1†source】.

2. **Histórico e Desenvolvimento dos LLMs:** Uma revisão histórica que vai das primeiras tentativas de criar regras linguísticas firmes até os avanços recentes em deep learning e modelos como BERT e GPT【4:1†source】.

3. **Aplicações dos LLMs:** Discute vários usos práticos dos LLMs, incluindo geração de conteúdo, análise de sentimento, classificação e agrupamento de textos, tradução de idiomas, e sumarização【4:4†source】.

4. **Aspectos Técnicos e Teóricos:** Descrição detalhada de como os LLMs funcionam, os princípios por trás de sua operação, e orientações para começar a usá-los【4:0†source】.

5. **Considerações Futuras e